# Connect to API

from https://www.analyticsvidhya.com/blog/2020/08/analysing-streaming-tweets-with-python-and-postgresql/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
import json 
import datetime
import time
import seaborn as sns

import psycopg2
%load_ext autoreload
%autoreload 2

In [ ]:
from auth_pg import *
from auth_ap import *

In [ ]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

In [ ]:
# Using the API object to get tweets from your timeline, and storing it in a variable called public_tweets
public_tweets = api.home_timeline()
# foreach through all tweets pulled
for tweet in public_tweets:
   # printing the text stored inside the tweet object
   print (tweet.text)

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, time_limit= 30):
        self.start_time = time.time()
        self.limit = time_limit
        super(MyStreamListener, self).__init__()
    
    def on_connect(self):
        print("Connected to Twitter API.")
        
    def on_status(self, status):
        
        # User info
        user_id = status.user.id
        username = status.user.name
        followers_count = status.user.followers_count
        friends_count = status.user.friends_count
        
        user_info = (user_id, username, followers_count, friends_count)
        
        # tweet info
        tweet_id = status.id
        created_at = status.created_at
        favorite_count = status.favorite_count
        source_device = status.source
         
        # Retweet count
        retweet_count = status.retweet_count
        # Language
        lang = status.lang
        
        # Tweet
        if status.truncated == True:
            tweet = status.extended_tweet['full_text']
            hashtags = status.extended_tweet['entities']['hashtags']
        else:
            tweet = status.text
            hashtags = status.entities['hashtags']
        
        tweet_info = (tweet_id, tweet, retweet_count, favorite_count, source_device, created_at)
        # Read hastags
        hashtags = read_hashtags(hashtags)   

        # If tweet is not a retweet and tweet is in English
        if not hasattr(status, "retweeted_status") and lang=="en":
            # Connect to database
            dbConnect(user_info, tweet_info, hashtags)
            
        if (time.time() - self.start_time) > self.limit:
            
            print(time.time(), self.start_time, self.limit)
            return False
            
    def on_error(self, status_code):
        if status_code == 420:
            # Returning False in on_data disconnects the stream
            return False

In [ ]:
# Insert Tweet data into database
def dbConnect(user_info, tweet_info, hashtags):
    user_id, username, followers_count, friends_count = user_info
    tweet_id, tweet, retweet_count, favorite_count, source_device, created_at = tweet_info
    
    database = 'twitterDB'
    # Connection to database server
    conn = psycopg2.connect(host= host,
                        database= database,
                        port= port,
                        user= user,
                        password= password)
    
    cur = conn.cursor()

    # insert user information
    command = '''INSERT INTO TwitterUser (user_id, user_name, followers_count, friends_count) 
                VALUES (%s,%s,%s,%s) ON CONFLICT
                 (User_Id) DO NOTHING;'''
    cur.execute(command,(user_id, username, followers_count, friends_count))

    # insert tweet information
    command = '''INSERT INTO TwitterTweet (tweet_id, user_id, tweet, retweet_count, favourite_count, source_device, created_at) 
                VALUES (%s,%s,%s,%s,%s,%s,%s);'''
    cur.execute(command,(tweet_id, user_id, tweet, retweet_count, favorite_count, source_device, created_at))
    
    # insert entity information
    for i in range(len(hashtags)):
        hashtag = hashtags[i]
        command = '''INSERT INTO TwitterEntity (tweet_id, hashtag) VALUES (%s,%s);'''
        cur.execute(command,(tweet_id, hashtag))
    
    # Commit changes
    conn.commit()
    
    # Disconnect
    cur.close()
    conn.close()

In [ ]:
# Extract hashtags
def read_hashtags(tag_list):
    hashtags = []
    for tag in tag_list:
        hashtags.append(tag['text'])
    return hashtags

In [ ]:
# Connecting to the Database
def DbConnect(query):
    
    conn = psycopg2.connect(host= host,
                        database= database,
                        port= port,
                        user= user,
                        password= password)
    curr = conn.cursor()
    
    curr.execute(query)
    
    rows = curr.fetchall()
    curr.close()
    conn.close()
    return rows

## Create table

In [ ]:
# Table creation
commands = (# Table 1
            '''Create Table TwitterUser(User_Id BIGINT PRIMARY KEY, 
                                        User_Name TEXT,
                                        Description TEXT,
                                        Followers_Count INT,
                                        Friends_Count INT
                                        );''',
            # Table 2
            '''Create Table TwitterTweet(Tweet_Id BIGINT PRIMARY KEY,
                                         User_Id BIGINT,
                                         Tweet TEXT,
                                         Retweet_Count INT,
                                         Favourite_Count INT,
                                         Source_Device TEXT,
                                         Created_At TEXT,
                                         CONSTRAINT fk_user
                                             FOREIGN KEY(User_Id)
                                                 REFERENCES TwitterUser(User_Id));''',
            # Table 3
            '''Create Table TwitterEntity(Id SERIAL PRIMARY KEY,
                                         Tweet_Id BIGINT,
                                         Hashtag TEXT,
                                         CONSTRAINT fk_user
                                             FOREIGN KEY(Tweet_Id)
                                                 REFERENCES TwitterTweet(Tweet_Id));''')

In [ ]:
# Connection to database server
database = 'twitterDB'
conn = psycopg2.connect(host= host,
                        database= database,
                        port= port,
                        user= user,
                        password= password)

In [ ]:
# Create cursor to execute SQL commands
cur = conn.cursor()

# Execute SQL commands
for command in commands:
    # Create tables
    cur.execute(command)

# Close communication with server
conn.commit()
cur.close()
conn.close()

## Obtain tweets

In [ ]:
# Streaming tweets
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener,
                        tweet_mode="extended")

myStream.filter(track=['#covid'])

In [ ]:
# Create 
data_tweet = DbConnect("SELECT User_Id, Tweet_Id, Tweet FROM TwitterTweet;")

df_tweet = pd.DataFrame(columns=['User_Id','Tweet_Id','Clean_Tweet'])

for data in data_tweet:
    index = len(df_tweet)
    df_tweet.loc[index,'User_Id'] = data[0]
    df_tweet.loc[index,'Tweet_Id'] = data[1]
    df_tweet.loc[index,'Clean_Tweet'] = data[2]
    
df_tweet.head()

In [ ]:
data_tweet = DbConnect("SELECT * FROM TwitterUser;")

In [ ]:
data_tweet

In [ ]:
conn.close()